In [1]:
import os
import json
import shutil
from sklearn.model_selection import train_test_split
from pycocotools.coco import COCO
from PIL import Image
from glob import glob
import pandas as pd
from tqdm import tqdm
from ultralytics import YOLO

각 알약 조합 별, 모든 알약에 대해 정보가 존재하는 이미지들을 구한다.

In [2]:
images=sorted(os.listdir('./ai06-level1-project/train_images'))
annot_root='./ai06-level1-project/train_annotations'
annots_t=sorted(os.listdir(annot_root)) # 알약 조합 목록
annots_list=[]
images_list=[]
# 각 알약 조합에 대해
for annot_set in annots_t:
    # 약 목록을 구한다.
    pill_list=sorted(os.listdir(os.path.join(annot_root,annot_set)))
    # 첫 약이 가지는 json 파일 목록 (각 이미지에 대한 json 파일 목록)
    base_images=sorted(os.listdir(os.path.join(annot_root,annot_set,pill_list[0])))
    # 각 약에 대해
    for pill in pill_list:
        # 각 약이 어떤 이미지에 대한 annotation을 갖는지에 대한 list
        imgs=sorted(os.listdir(os.path.join(annot_root,annot_set,pill)))
        # 이번 약이 갖지 않는 json 파일들은 제거
        base_images=[image for image in base_images if image in imgs]
    # 모든 약에 대해, 공통으로 json 파일이 있는 이미지의 이름만 저장
    for image in base_images:
        annots_list.append(image[:-5])
# 
for img in images:
    images_list.append(img[:-4])
len(annots_list), annots_list[0], len(images_list),images_list[0]

(225,
 'K-001900-016548-019607-029451_0_2_0_2_70_000_200',
 661,
 'K-001900-010224-016551-031705_0_2_0_2_70_000_200')

전체 파일 중, annotation이 이미지의 모든 알약에 대해 있는 이미지들의 이름만 추출

In [3]:
images_set=set(images_list)
annots_set=set(annots_list)
# 이미지에 대해, 모든 알약의 annotation이 있는 경우의 이름들을 모음
available_files=sorted(list(images_set.intersection(annots_set)))
len(available_files)

224

In [4]:
with open('./annots_without_image.txt','w') as f:
    for annot in annots_list:
        if annot not in images_list:
            f.write(str(annot)+'\n')

In [5]:
with open('./images_without_annot.txt','w') as f:
    for image in images_list:
        if image not in annots_list:
            f.write(str(image)+'\n')

In [6]:
for f in available_files:
    print(f)
    break

K-001900-016548-019607-029451_0_2_0_2_70_000_200


이미지 단위로 annotation 파일들을 결합합니다.

In [7]:
#json 파일 통합

base_path = "./ai06-level1-project"
ann_root = os.path.join(base_path, "train_annotations")

output_path = os.path.join(base_path, "train_annots")
os.makedirs(output_path, exist_ok=True)

image_id_offset = 0
annotation_id_offset = 0
category_set = set()

# 알약 조합 별 폴더 명
for root in sorted(os.listdir(ann_root)):

    # 현재 조합의 알약 종류
    pills=sorted(os.listdir(os.path.join(ann_root,root)))

    # 현재 조합의 이미지 목록
    images_list=[]
    images = os.listdir(os.path.join(ann_root,root,pills[0]))

    if len(images)==0:
        continue

    # 각 이미지 별 JSON 파일 생성
    for img in images:

        images=[]
        annotations=[]
        categories=[]

        # 대응하는 이미지가 없거나, 일부 알약에 대해 누락되어 있다 annotation이 누락된 이미지면 continue
        if img[:-5] not in available_files:
            continue
        # 각 알약에 대해서
        for pill in pills:
            # 해당 이미지의 json 파일을 엽니다.
            with open(str(os.path.join(ann_root,root,pill,img)), "r", encoding="utf-8") as f:
                pill_data = json.load(f)
                if len(images)==0:
                    images.extend(pill_data["images"])
                annotations.extend(pill_data["annotations"])
                categories.extend(pill_data["categories"])
        output_coco={
            "images": images,
            "annotations": annotations,
            "categories": categories
        }
        with open(os.path.join(output_path,img), "w", encoding="utf-8") as f:
            json.dump(output_coco, f, ensure_ascii=False, indent=4)

이제 모든 이미지의 json 파일을 결합합니다.

In [8]:
output_all_coco = {
    "images": [],
    "annotations": [],
    "categories": []
}
files = os.listdir(os.path.join(base_path, "train_annots"))

for file in files:
    with open(os.path.join(base_path, "train_annots", file), "r", encoding="utf-8") as f:
        data=json.load(f)
        for image in data['images']:
            output_all_coco['images'].append(image)
        for annot in data['annotations']:
            output_all_coco['annotations'].append(annot)
        for cat in data['categories']:
            output_all_coco['categories'].append(cat)
            
save_path = os.path.join(base_path, "train.json")
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(output_all_coco, f, ensure_ascii=False, indent=4)

print("train.json 생성됨:", save_path)

train.json 생성됨: ./ai06-level1-project/train.json


In [9]:
len(output_all_coco['categories'])

746

In [10]:
BASE = r"./ai06-level1-project"
ANN_FILE = os.path.join(BASE, "train.json")
with open(ANN_FILE, "r", encoding="utf-8") as f:
    dataset = json.load(f)

In [13]:
#YOLO용으로 데이터 전처리

BASE = r"./ai06-level1-project/"
IMG_DIR = os.path.join(BASE, "train_output")
ANN_FILE = os.path.join(BASE, "train.json")
TEST_IMG_DIR = os.path.join(BASE, "test_images")

OUT_DIR = os.path.join(BASE, "yolo_dataset")

os.makedirs(os.path.join(OUT_DIR, "images/train"), exist_ok=True)
os.makedirs(os.path.join(OUT_DIR, "images/val"), exist_ok=True)
os.makedirs(os.path.join(OUT_DIR, "labels/train"), exist_ok=True)
os.makedirs(os.path.join(OUT_DIR, "labels/val"), exist_ok=True)

with open(ANN_FILE, "r", encoding="utf-8") as f:
    dataset = json.load(f)

coco = COCO()
coco.dataset = dataset
coco.createIndex()

img_ids = list(coco.imgs.keys())

train_ids, val_ids = train_test_split(img_ids, test_size=0.2, random_state=42)


def convert_to_yolo_bbox(box, img_w, img_h):
    x, y, w, h = box
    cx = (x + w/2) / img_w
    cy = (y + h/2) / img_h
    w /= img_w
    h /= img_h
    return cx, cy, w, h


def process_image(img_id, split="train"):

    img_info = coco.loadImgs(img_id)[0]
    file_name = img_info["file_name"]
    width, height = img_info["width"], img_info["height"]

    src_img_path = os.path.join(IMG_DIR, file_name)
    dst_img_path = os.path.join(OUT_DIR, f"images/{split}/{file_name}")

    if os.path.exists(src_img_path):
        shutil.copy(src_img_path, dst_img_path)
    else:
        print("이미지 없음:", src_img_path)
        return

    label_path = os.path.join(OUT_DIR, f"labels/{split}/{file_name.replace('.png', '.txt')}")

    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)

    with open(label_path, "w", encoding="utf-8") as f:
        for ann in anns:
            category_id = ann["category_id"]
            yolo_class = list(coco.cats.keys()).index(category_id)
            bbox = ann["bbox"]
            yolo_box = convert_to_yolo_bbox(bbox, width, height)

            f.write(f"{yolo_class} {' '.join([str(round(v, 6)) for v in yolo_box])}\n")


for img_id in train_ids:
    process_image(img_id, split="train")

for img_id in val_ids:
    process_image(img_id, split="val")

print("YOLO dataset 생성 완료")


yaml_path = os.path.join(OUT_DIR, "data.yaml")
num_classes = len(coco.cats)
names = [coco.cats[k]["name"] for k in sorted(coco.cats.keys())]

with open(yaml_path, "w", encoding="utf-8") as f:
    f.write(f"path: {OUT_DIR}\n")
    f.write("train: images/train\n")
    f.write("val: images/val\n\n")
    f.write(f"nc: {num_classes}\n")
    f.write(f"names: {names}\n")

print("data.yaml 파일 생성 완료")

creating index...
index created!
YOLO dataset 생성 완료
data.yaml 파일 생성 완료
